<a href="https://colab.research.google.com/github/arthursl12/POC1/blob/main/POC2_FD002_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
%pip install scikeras;
%pip install -U tensorflow-addons;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 7.6 MB/s 


In [ ]:
import os
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob
import datetime

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.base import BaseEstimator,RegressorMixin

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

from tensorflow.keras.metrics import RootMeanSquaredError as RMSE

from keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.optimizers.schedules import ExponentialDecay

import tensorflow_addons as tfa
from tensorflow_addons.metrics import RSquare as R2



from scikeras.wrappers import KerasRegressor

In [ ]:
sns.set_palette('colorblind')

In [ ]:
# Reproducibility
seed = 42
os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
tf.config.experimental.enable_op_determinism()

# Data Preparation

In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    IN_COLAB = True
    folder="/content/CMaps/"

    # Dataset Download 
    os.system('git clone https://github.com/arthursl12/dataset_2')
    os.system('mv /content/dataset_2/CMaps /content/CMaps')
    os.system('mv /content/dataset_2/data_processing /content/data_processing')
    os.system('rm -rf dataset_2')
else:
    print('Not running on CoLab')
    IN_COLAB = False
    folder="CMaps/"
    %cd dataset_2/

Running on CoLab


In [ ]:
from data_processing.processing import DatasetProcessing
from data_processing.training import HyperparameterSearch, reclipper_scorer
from data_processing.eval import Evaluation

In [ ]:
proc = DatasetProcessing()

## Data Integration

The data are provided as a zip-compressed text file with 26 columns of numbers, separated by spaces. Each row is a snapshot of data taken during a single operational cycle, each column is a different variable. The columns correspond to:  

1) unit number   
2) time, in cycles  
3) operational setting 1  
4) operational setting 2  
5) operational setting 3    
6) sensor measurement 1    
7) sensor measurement 2  
...  
26) sensor measurement 20


There are 6 conditions (or combinations) which the 3 operational settings can take.  
Condition 1: Altitude = 0, Mach Number = 0, TRA = 100  
Condition 2: Altitude = 10, Mach Number = 0.25, TRA = 100  
Condition 3: Altitude = 20, Mach Number = 0.7 TRA = 100  
Condition 4: Altitude = 25, Mach Number = 0.62, TRA = 60  
Condition 5: Altitude = 35 Mach Number = 0.84, TRA = 100  
Condition 6: Altitude = 42, Mach Number = 0.84, TRA = 100  
  
There is slight variation in all these conditions so you may get numbers like 24.453 instead of 25 exactly.

FD001: Condition 1 only  
FD002: Mix of all the conditions  
FD003: Condition 1 only  
FD004: Mix of all conditions  


In [ ]:
index_cols, settings_cols, sensors_cols, cols = proc.column_names()
train, test, y_test = proc.read_dataset(2,folder=folder)
train

,unit_number,time,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,1,1,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,...,183.06,2387.72,8048.56,9.3461,0.02,334,2223,100.00,14.73,8.8071
1,1,2,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,...,130.42,2387.66,8072.30,9.3774,0.02,330,2212,100.00,10.41,6.2665
2,1,3,24.9988,0.6218,60.0,462.54,537.31,1256.76,1047.45,7.05,...,164.22,2028.03,7864.87,10.8941,0.02,309,1915,84.93,14.08,8.6723
3,1,4,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,...,130.72,2387.61,8068.66,9.3528,0.02,329,2212,100.00,10.59,6.4701
4,1,5,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,7.05,...,164.31,2028.00,7861.23,10.8963,0.02,309,1915,84.93,14.13,8.5286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53754,260,312,20.0037,0.7000,100.0,491.19,608.79,1495.60,1269.51,9.35,...,314.05,2389.02,8169.64,9.3035,0.03,369,2324,100.00,24.36,14.5189
53755,260,313,10.0022,0.2510,100.0,489.05,605.81,1514.32,1324.12,10.52,...,371.22,2388.42,8245.36,8.7586,0.03,374,2319,100.00,28.10,16.9454
53756,260,314,25.0041,0.6200,60.0,462.54,537.48,1276.24,1057.92,7.05,...,163.74,2030.33,7971.25,11.0657,0.02,310,1915,84.93,14.19,8.5503
53757,260,315,25.0033,0.6220,60.0,462.54,537.84,1272.95,1066.30,7.05,...,164.37,2030.35,7972.47,11.0537,0.02,311,1915,84.93,14.05,8.3729


## Preprocessing

### Test Set Transformation 
Test set has samples for all cycles, but has annotations only for last one

In [ ]:
test.shape, y_test.shape

((33991, 26), (259, 1))

In [ ]:
test_last = proc.transform_test_keep_setting(test)
test_last.head()

,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,s_5,s_6,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,10.0076,0.2501,100.0,489.05,605.42,1515.00,1325.07,10.52,15.50,393.58,...,370.87,2388.32,8167.06,8.7456,0.03,371,2319,100.0,28.30,17.0934
1,0.0018,0.0000,100.0,518.67,642.67,1591.67,1418.17,14.62,21.61,553.36,...,521.10,2388.12,8138.12,8.4248,0.03,393,2388,100.0,38.82,23.3463
2,35.0015,0.8412,100.0,449.44,555.86,1370.62,1135.59,5.48,8.00,194.58,...,183.11,2388.07,8071.23,9.3094,0.02,332,2223,100.0,14.75,8.9589
3,20.0032,0.7000,100.0,491.19,607.99,1487.94,1257.49,9.35,13.66,334.39,...,314.88,2388.12,8062.39,9.2349,0.02,365,2324,100.0,24.22,14.6814
4,42.0055,0.8400,100.0,445.00,550.81,1358.95,1140.34,3.91,5.72,138.42,...,130.82,2389.06,8140.94,9.3964,0.02,333,2212,100.0,10.34,6.3601


In [ ]:
X_test = test_last

### Remaining Useful Life (RUL)

In [ ]:
train = proc.add_remaining_useful_life_linear(train)
train[index_cols+['RUL']].head()

,unit_number,time,RUL
0,1,1,148
1,1,2,147
2,1,3,146
3,1,4,145
4,1,5,144


## Attributes and target separation

In [ ]:
X_train, y_train = proc.X_y_train_divide_with_settings(train)

In [ ]:
y_train.head()

,RUL
0,148
1,147
2,146
3,145
4,144


In [ ]:
X_train.head()

,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,s_5,s_6,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,8.00,194.64,...,183.06,2387.72,8048.56,9.3461,0.02,334,2223,100.00,14.73,8.8071
1,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,5.71,138.51,...,130.42,2387.66,8072.30,9.3774,0.02,330,2212,100.00,10.41,6.2665
2,24.9988,0.6218,60.0,462.54,537.31,1256.76,1047.45,7.05,9.02,175.71,...,164.22,2028.03,7864.87,10.8941,0.02,309,1915,84.93,14.08,8.6723
3,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,5.71,138.46,...,130.72,2387.61,8068.66,9.3528,0.02,329,2212,100.00,10.59,6.4701
4,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,7.05,9.03,175.05,...,164.31,2028.00,7861.23,10.8963,0.02,309,1915,84.93,14.13,8.5286


## Training and Evaluation functions

In [ ]:
eval = Evaluation()

In [ ]:
search = HyperparameterSearch()

# MLP Construction

## Callbacks

In [ ]:
# Early Stopping Callback
es = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                      patience=5, restore_best_weights=True)

In [ ]:
# Printing Callback
def printLog(epoch, logs):
    print(
        f"E {epoch+1}\t: loss={logs['loss']:.3f}, "+
        f"rmse={logs['root_mean_squared_error']:.3f}, "+
        f"r2={logs['r_square']:.3f}; "+
        f"v_loss={logs['val_loss']:.3f}, "+
        f"v_rmse={logs['val_root_mean_squared_error']:.3f}, "+
        f"v_r2={logs['val_r_square']:.3f}; "
    )

printerCallback = LambdaCallback(on_epoch_end=printLog)

## Constants

In [ ]:
# X_train must include indices
train3 = train.copy()
X_train_ = train3.drop(columns=[index_cols[1]]+["RUL"])

In [ ]:
INPUT_SHAPE = 0

## Wrapper

In [ ]:
from sys import is_finalizing
from sklearn.metrics import r2_score

class MLPWrapperRegressor(BaseEstimator,RegressorMixin):
    def __init__(self, basemodel=None, clip_y=-1,
                 include_settings=False, poly_degree=1,
                 scaler=StandardScaler()):
        # Base parameters
        self.basemodel = basemodel
        self.clip_y = clip_y
        self.poly_degree = poly_degree
        self.include_settings = include_settings

        # Column indexers
        self.feature_cols = sensors_cols
        if(include_settings):
            self.feature_cols = settings_cols + self.feature_cols
        self.base_feature_cols = self.feature_cols

        # Scaler and PolyFeatures transformers
        self.scaler = scaler
        self.polyft = PolynomialFeatures(degree=self.poly_degree, 
                                         include_bias=False)

    def clean_cols(self,df):
        if("unit_number" in df.columns): del df["unit_number"]
        if("unit_number" in df.columns): del df["unit_number"]
        if("time" in df.columns): del df["time"]
        if((not self.include_settings)): 
            for col in settings_cols:
                if(col in df.columns): del df[col]
        return df

    def fit(self, X=None, y=None):
        # Drop irrelevant column
        data = pd.DataFrame(X).copy()
        data = self.clean_cols(data)
        
        # Apply polynomial features and add them to the dataframe
        transf = self.polyft.fit_transform(data)
        data = pd.DataFrame(transf, columns=
                              self.polyft.get_feature_names_out())
        self.feature_cols = list(self.polyft.get_feature_names_out())

        # Scale the data
        data = self.scaler.fit_transform(data)
        data = pd.DataFrame(data, 
                            columns=self.scaler.get_feature_names_out())
        X_train = data.copy()
        
        # Clip and transform labels
        data2 = pd.DataFrame(y, columns=["RUL"]).copy()
        if (self.clip_y > 0):
            data2["RUL"].clip(upper=self.clip_y, inplace=True)
        y_train = data2

        # Update input shape for future use
        global INPUT_SHAPE
        INPUT_SHAPE = X_train.shape[1]

        # Fit model
        self.basemodel.fit(X_train,y_train)
        return self
        
    def predict(self, X=None):
        # Perform transformation, if not done
        if (len(X.columns) != len(self.feature_cols)):
            X_train = self.transform_features(X)
        else:
            X_train = X
        return self.basemodel.predict(X_train)

    def transform_features(self, df):
        # Drop irrelevant column
        data = pd.DataFrame(df).copy()
        data = self.clean_cols(data)
        
        # Apply polynomial features and add them to the dataframe
        transf = self.polyft.transform(data)
        data = pd.DataFrame(transf, columns=
                              self.polyft.get_feature_names_out())
        # self.feature_cols = list(self.polyft.get_feature_names_out())

        # Scale the data
        data = self.scaler.transform(data)
        data = pd.DataFrame(data, 
                            columns=self.scaler.get_feature_names_out())
        return data

    def score(self, X, y, sample_weight=None):
        X_test = self.transform_features(X)

        # Clip and transform labels
        data2 = pd.DataFrame(y, columns=["RUL"]).copy()
        if (self.clip_y > 0):
            data2["RUL"].clip(upper=self.clip_y, inplace=True)
        y_test = data2

        # Predict on test data
        y_pred = self.predict(X_test)
        return r2_score(y_test, y_pred, sample_weight=sample_weight)

## Test Data

In [ ]:
def scale_test(test,model):
    test2 = pd.DataFrame(test)
    test2 = model.clean_cols(test2)

    # Apply polynomial features
    transf = model.polyft.transform(test2)
    test2 = pd.DataFrame(transf, 
                         columns=model.polyft.get_feature_names_out())

    # Scale the data (with train data parameters)
    test2 = model.scaler.transform(test2)
    test2 = pd.DataFrame(test2, 
                         columns=model.polyft.get_feature_names_out())
    return test2

## Constructor

In [ ]:
def create_model(optim=Adam, learning_rate=1e-3, 
                 layer1=32  , activation1="tanh"    , dropout1=0.1,
                 layer2=None, activation2="tanh"    , dropout2=0.1,
                 layer3=None, activation3="tanh"    , dropout3=0.1,
                 layer4=None, activation4="tanh"    , dropout4=0.1,
                 print_summary=False, loss='mean_squared_error',
                 metrics=[tf.keras.metrics.MeanSquaredError()]):
    model = Sequential()

    # Fully Connected Layer
    model.add(Dense(layer1, input_dim=INPUT_SHAPE, activation=activation1))
    model.add(Dropout(dropout1))

    # Additional hidden layers
    if(layer2 is not None):
        model.add(Dense(layer2, activation=activation2))
        model.add(Dropout(dropout2))
        if (layer3 is not None):
            model.add(Dense(layer3, activation=activation3))
            model.add(Dropout(dropout3))
            if (layer4 is not None):
                model.add(Dense(layer4, activation=activation4))
                model.add(Dropout(dropout4))

    # Output Layer
    model.add(Dense(1))

    model.compile(loss=loss, optimizer=optim(learning_rate=learning_rate), 
                  metrics=metrics)
    
    if(print_summary): model.summary()
    return model

# MLP-1 (REDO Some)

## Linear RUL (REDO)

Score: 0.581174610036398  
Test: 0.
```
('basemodel__batch_size', 117),
             ('basemodel__epochs', 42),
             ('basemodel__model__activation1', 'tanh'),
             ('basemodel__model__dropout1', 0.1),
             ('basemodel__model__layer1', 512),
             ('basemodel__model__learning_rate', 0.0034318227561347635),
             ('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
             ('basemodel__validation_split', 0.1),
             ('scaler', StandardScaler())
```

In [ ]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=117,
                           epochs=42,
                           model__activation1='tanh',
                           model__dropout1=0.1, 
                           model__layer1=512, 
                           model__learning_rate=0.0034318227561347635,
                           model__optim=Adam,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=117, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f9bff274110>, <keras.callbacks.LambdaCallback object at 0x7f9bff278410>], epochs=42, model=<function create_model at 0x7f9bff2b4dd0>, model__activation1='tanh', model__dropout1=0.1, model__layer1=512, model__learning_rate=0.0034318227561347635, model__loss='mse', model__metrics=[<keras.metrics.metrics.RootMeanSquaredError object at 0x7f9bf00cde90>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f9bf00c8650>], model__optim=<class 'keras.optimizers.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1, verbose=0))

In [ ]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               11264     
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 513       
                                                                 
Total params: 11,777
Trainable params: 11,777
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=5468.074, rmse=73.946, r2=-0.169; v_loss=3801.744, v_rmse=61.658, v_r2=0.333; 
E 2	: loss=2596.202, rmse=50.953, r2=0.445; v_loss=2822.301, v_rmse=53.125, v_r2=0.505; 
E 3	: loss=2211.476, rmse=47.026, r2=0.527; v_loss=2521.757, v_rmse=50.217, v_r2=0.558; 
E 4	: loss=2126.377, rmse=46.113, r2=0.545; v_lo

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.639,RMSE=-32.310
Finished: 2022-10-30 11:12:17.444521


## Non-Linear RUL

Score: 0.8184403480530235  
Test: 0.
```
('basemodel__batch_size', 32),
('basemodel__epochs', 40),
('basemodel__model__activation1', 'sigmoid'),
('basemodel__model__dropout1', 0.1),
('basemodel__model__layer1', 512),
('basemodel__model__learning_rate', 0.01),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.1),
('clip_y', 80),
('include_settings', True),
('scaler', StandardScaler())
```

In [ ]:
CLIP=80

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=MinMaxScaler(), poly_degree=1,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=32,
                           epochs=250,
                           model__activation1='sigmoid',
                           model__dropout1=0.1, 
                           model__layer1=512, 
                           model__learning_rate=0.01,
                           model__optim=Adam,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f9bff274110>, <keras.callbacks.LambdaCallback object at 0x7f9bff278410>], epochs=250, model=<function create_model at 0x7f9bff2b4dd0>, model__activation1='sigmoid', model__dropout1=0.1, model__layer1=512, model__learning_rate=0.01, model__loss='mse', model__metrics=[<keras.metrics.metrics.RootMeanSquaredError object at 0x7f9b8004ac90>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f9b80044290>], model__optim=<class 'keras.optimizers.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1, verbose=0),
                    clip_y=80, include_settings=True, scaler=MinMaxScaler())

In [ ]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 512)               12800     
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 dense_9 (Dense)             (None, 1)                 513       
                                                                 
Total params: 13,313
Trainable params: 13,313
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=548.853, rmse=23.428, r2=0.080; v_loss=407.656, v_rmse=20.190, v_r2=0.299; 
E 2	: loss=351.107, rmse=18.738, r2=0.412; v_loss=261.609, v_rmse=16.174, v_r2=0.550; 
E 3	: loss=241.596, rmse=15.543, r2=0.595; v_loss=198.407, v_rmse=14.086, v_r2=0.659; 
E 4	: loss=216.622, rmse=14.718, r2=0.637; v_loss=231

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.829,RMSE=-11.565
Finished: 2022-10-30 11:39:04.750040


## PolyFeatures + Linear RUL (REDO)


Score: 0.5760357993360262   
Test: 0.
```
('basemodel__batch_size', 32),
             ('basemodel__epochs', 50),
             ('basemodel__model__activation1', 'elu'),
             ('basemodel__model__dropout1', 0.5527766101177216),
             ('basemodel__model__layer1', 61),
             ('basemodel__model__learning_rate', 0.008905200618475653),
             ('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
             ('basemodel__validation_split', 0.168565014223427),
             ('poly_degree', 3),
             ('scaler', StandardScaler())
```

In [ ]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=3,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=32,
                           epochs=50,
                           model__activation1='elu',
                           model__dropout1=0.5527766101177216, 
                           model__layer1=61, 
                           model__learning_rate=0.008905200618475653,
                           model__optim=Adam,
                           validation_split=0.168565014223427, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f9bff274110>, <keras.callbacks.LambdaCallback object at 0x7f9bff278410>], epochs=50, model=<function create_model at 0x7f9bff2b4dd0>, model__activation1='elu', model__dropout1=0.5527766101177216, model__layer1=61, model__learning_rate=0.008905200618475653, model__loss='mse', model__metrics=[<keras.metrics.metrics.RootMeanSquaredError object at 0x7f9b6fe77350>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f9b6fe77510>], model__optim=<class 'keras.optimizers.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.168565014223427, verbose=0),
                    include_settings=True, poly_degree=3)

In [ ]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 61)                178425    
                                                                 
 dropout_5 (Dropout)         (None, 61)                0         
                                                                 
 dense_11 (Dense)            (None, 1)                 62        
                                                                 
Total params: 178,487
Trainable params: 178,487
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=4066.785, rmse=63.771, r2=0.144; v_loss=3132.908, v_rmse=55.972, v_r2=0.368; 
E 2	: loss=3205.536, rmse=56.617, r2=0.325; v_loss=2393.167, v_rmse=48.920, v_r2=0.518; 
E 3	: loss=3086.243, rmse=55.554, r2=0.350; v_loss=2410.662, v_rmse=49.098, v_r2=0.514; 
E 4	: loss=3028.551, rmse=55.032, r2=0.362; v

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.669,RMSE=-30.927
Finished: 2022-10-30 11:44:42.779856


## PolyFeatures + Non-Linear RUL


Score: 0.746401337342355  
Test: 0.
```
('basemodel__batch_size', 32),
('basemodel__epochs', 50),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__dropout1', 0.10000000188658846),
('basemodel__model__layer1', 512),
('basemodel__model__learning_rate', 0.01),
('basemodel__model__optim',
keras.optimizers.optimizer_v2.rmsprop.RMSprop),
('basemodel__validation_split', 0.9),
('clip_y', 80),
('include_settings', True),
('poly_degree', 2),
('scaler', StandardScaler())
```

In [ ]:
CLIP=80

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=2,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=32,
                           epochs=50,
                           model__activation1='tanh',
                           model__dropout1=0.10000000188658846, 
                           model__layer1=512, 
                           model__learning_rate=0.01,
                           model__optim=RMSprop,
                           validation_split=0.9, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=32, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f9bff274110>, <keras.callbacks.LambdaCallback object at 0x7f9bff278410>], epochs=50, model=<function create_model at 0x7f9bff2b4dd0>, model__activation1='tanh', model__dropout1=0.10000000188658846, model__layer1=512, model__learning_rate=0.01, model__loss='mse', model__metrics=[<keras.metrics.metrics.RootMeanSquaredError object at 0x7f9b6fbbbf50>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f9b6fb43b90>], model__optim=<class 'keras.optimizers.optimizer_v2.rmsprop.RMSprop'>, print_summary=True, validation_split=0.9, verbose=0),
                    clip_y=80, include_settings=True, poly_degree=2)

In [ ]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               166400    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 1)                 513       
                                                                 
Total params: 166,913
Trainable params: 166,913
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=1663.746, rmse=40.789, r2=-1.734; v_loss=581.357, v_rmse=24.111, v_r2=0.021; 
E 2	: loss=509.678, rmse=22.576, r2=0.162; v_loss=391.558, v_rmse=19.788, v_r2=0.341; 
E 3	: loss=347.597, rmse=18.644, r2=0.429; v_loss=250.452, v_rmse=15.826, v_r2=0.578; 
E 4	: loss=276.094, rmse=16.616, r2=0.546; v_loss

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.734,RMSE=-14.426
Finished: 2022-10-30 11:47:54.219725


# MLP-2 

## Linear RUL

Score: 0.5812000406687549  
Test: 0.
```
('basemodel__batch_size', 502),
('basemodel__epochs', 19),
('basemodel__model__activation1', 'relu'),
('basemodel__model__activation2', 'elu'),
('basemodel__model__dropout1', 0.537010358774788),
('basemodel__model__dropout2', 0.639760677232973),
('basemodel__model__layer1', 507),
('basemodel__model__layer2', 151),
('basemodel__model__learning_rate', 0.0055564604418504495),
('basemodel__model__optim',
keras.optimizers.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.1),
('include_settings', True),
('scaler', StandardScaler())
```

In [ ]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=502,
                           epochs=19,
                           model__activation1='relu',
                           model__activation2='elu',
                           model__dropout1=0.537010358774788, 
                           model__dropout2=0.639760677232973, 
                           model__layer1=507, 
                           model__layer2=151, 
                           model__learning_rate=0.0055564604418504495,
                           model__optim=Adam,
                           validation_split=0.1, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=502, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f9bff274110>, <keras.callbacks.LambdaCallback object at 0x7f9bff278410>], epochs=50, model=<function create_model at 0x7f9bff2b4dd0>, model__activation1='relu', model__activation2='elu', model__dropout1=0.537010358774788, model__dropout2=0.63976067723...51, model__learning_rate=0.0055564604418504495, model__loss='mse', model__metrics=[<keras.metrics.metrics.RootMeanSquaredError object at 0x7f9b6fdc6610>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f9b6ffb9890>], model__optim=<class 'keras.optimizers.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1, verbose=0),
                    include_settings=True)

In [ ]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 507)               12675     
                                                                 
 dropout_9 (Dropout)         (None, 507)               0         
                                                                 
 dense_18 (Dense)            (None, 151)               76708     
                                                                 
 dropout_10 (Dropout)        (None, 151)               0         
                                                                 
 dense_19 (Dense)            (None, 1)                 152       
                                                                 
Total params: 89,535
Trainable params: 89,535
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=5569.220, rmse=74.627, r2=-0.191; v_los

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.613,RMSE=-33.435
Finished: 2022-10-30 11:55:02.868973


## Non-Linear RUL

Score: 0.806487727981995  
Test: 0.
```
('basemodel__batch_size', 91),
('basemodel__epochs', 50),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'elu'),
('basemodel__model__dropout1', 0.8099281594197152),
('basemodel__model__dropout2', 0.3275285867415841),
('basemodel__model__layer1', 61),
('basemodel__model__layer2', 379),
('basemodel__model__learning_rate', 0.0017750683313794622),
('basemodel__model__optim', keras.optimizer_v2.rmsprop.RMSprop),
('basemodel__validation_split', 0.41651962978418944),
('clip_y', 80),
('include_settings', True),
('scaler', StandardScaler())
```

In [ ]:
CLIP=80

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=91,
                           epochs=250,
                           model__activation1='tanh',
                           model__activation2='elu',
                           model__dropout1=0.8099281594197152, 
                           model__dropout2=0.3275285867415841, 
                           model__layer1=61, 
                           model__layer2=379, 
                           model__learning_rate=0.0017750683313794622,
                           model__optim=RMSprop,
                           validation_split=0.41651962978418944, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=91, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f9bff274110>, <keras.callbacks.LambdaCallback object at 0x7f9bff278410>], epochs=250, model=<function create_model at 0x7f9bff2b4dd0>, model__activation1='tanh', model__activation2='elu', model__dropout1=0.8099281594197152, model__dropout2=0.3275285867...learning_rate=0.0017750683313794622, model__loss='mse', model__metrics=[<keras.metrics.metrics.RootMeanSquaredError object at 0x7f9b6e5509d0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f9b6e537ed0>], model__optim=<class 'keras.optimizers.optimizer_v2.rmsprop.RMSprop'>, print_summary=True, validation_split=0.41651962978418944, verbose=0),
                    clip_y=80)

In [ ]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 61)                1342      
                                                                 
 dropout_13 (Dropout)        (None, 61)                0         
                                                                 
 dense_24 (Dense)            (None, 379)               23498     
                                                                 
 dropout_14 (Dropout)        (None, 379)               0         
                                                                 
 dense_25 (Dense)            (None, 1)                 380       
                                                                 
Total params: 25,220
Trainable params: 25,220
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=1240.595, rmse=35.222, r2=-1.095; v_lo

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.811,RMSE=-12.159
Finished: 2022-10-30 12:09:13.792783


## PolyFeatures + Linear RUL

Score: 0.5749427963405077  
Test: 0.
```
('basemodel__batch_size', 108),
('basemodel__epochs', 50),
('basemodel__model__activation1', 'elu'),
('basemodel__model__activation2', 'elu'),
('basemodel__model__dropout1', 0.6853622426866657),
('basemodel__model__dropout2', 0.4181419652000947),
('basemodel__model__layer1', 392),
('basemodel__model__layer2', 51),
('basemodel__model__learning_rate', 0.007788469297925655),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.31679716488321474),
('include_settings', True),
('poly_degree', 3),
('scaler', StandardScaler())
```


In [ ]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=3,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=108,
                           epochs=50,
                           model__activation1='elu',
                           model__activation2='elu',
                           model__dropout1=0.6853622426866657, 
                           model__dropout2=0.4181419652000947, 
                           model__layer1=392, 
                           model__layer2=51, 
                           model__learning_rate=0.007788469297925655,
                           model__optim=Adam,
                           validation_split=0.31679716488321474, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=108, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f9bff274110>, <keras.callbacks.LambdaCallback object at 0x7f9bff278410>], epochs=50, model=<function create_model at 0x7f9bff2b4dd0>, model__activation1='elu', model__activation2='elu', model__dropout1=0.6853622426866657, model__dropout2=0.41814196520...l__learning_rate=0.007788469297925655, model__loss='mse', model__metrics=[<keras.metrics.metrics.RootMeanSquaredError object at 0x7f9b6e1a5190>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f9b6e1a5c10>], model__optim=<class 'keras.optimizers.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.31679716488321474, verbose=0),
                    poly_degree=3)

In [ ]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 392)               793408    
                                                                 
 dropout_15 (Dropout)        (None, 392)               0         
                                                                 
 dense_27 (Dense)            (None, 51)                20043     
                                                                 
 dropout_16 (Dropout)        (None, 51)                0         
                                                                 
 dense_28 (Dense)            (None, 1)                 52        
                                                                 
Total params: 813,503
Trainable params: 813,503
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=5311.123, rmse=72.877, r2=-0.081; v_

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.623,RMSE=-33.018
Finished: 2022-10-30 12:13:14.195920


## PolyFeatures + Non-Linear RUL


Score: 0.8061999136353725  
Test: 0.
``` 
('basemodel__batch_size', 163),
('basemodel__epochs', 32),
('basemodel__model__activation1', 'selu'),
('basemodel__model__activation2', 'relu'),
('basemodel__model__dropout1', 0.1203636705927238),
('basemodel__model__dropout2', 0.3108545284702169),
('basemodel__model__layer1', 18),
('basemodel__model__layer2', 364),
('basemodel__model__learning_rate', 0.009076852273996898),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.14877733495106543),
('clip_y', 87),
('include_settings', True),
('poly_degree', 2),
('scaler', MinMaxScaler())
```

In [ ]:
CLIP=87

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=MinMaxScaler(), poly_degree=2,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=163,
                           epochs=32,
                           model__activation1='selu',
                           model__activation2='relu',
                           model__dropout1=0.1203636705927238, 
                           model__dropout2=0.3108545284702169, 
                           model__layer1=18, 
                           model__layer2=364, 
                           model__learning_rate=0.009076852273996898,
                           model__optim=Adam,
                           validation_split=0.14877733495106543, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=163, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f9bff274110>, <keras.callbacks.LambdaCallback object at 0x7f9bff278410>], epochs=32, model=<function create_model at 0x7f9bff2b4dd0>, model__activation1='selu', model__activation2='relu', model__dropout1=0.1203636705927238, model__dropout2=0.310854528...', model__metrics=[<keras.metrics.metrics.RootMeanSquaredError object at 0x7f9b6a73f8d0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f9b6a73f950>], model__optim=<class 'keras.optimizers.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.14877733495106543, verbose=0),
                    clip_y=87, include_settings=True, poly_degree=2,
                    scaler=MinMaxScaler())

In [ ]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_29 (Dense)            (None, 18)                5850      
                                                                 
 dropout_17 (Dropout)        (None, 18)                0         
                                                                 
 dense_30 (Dense)            (None, 364)               6916      
                                                                 
 dropout_18 (Dropout)        (None, 364)               0         
                                                                 
 dense_31 (Dense)            (None, 1)                 365       
                                                                 
Total params: 13,131
Trainable params: 13,131
Non-trainable params: 0
_________________________________________________________________
E 1	: loss=834.728, rmse=28.892, r2=-0.133; v_los

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.770,RMSE=-14.709
Finished: 2022-10-30 12:15:21.011736


# MLP-3 

## Linear RUL

Score: 0.5854880998236767    
Test: 0.
```
('basemodel__batch_size', 200),
('basemodel__epochs', 35),
('basemodel__model__activation1', 'selu'),
('basemodel__model__activation2', 'sigmoid'),
('basemodel__model__activation3', 'sigmoid'),
('basemodel__model__dropout1', 0.38531894797761335),
('basemodel__model__dropout2', 0.2713124052618954),
('basemodel__model__dropout3', 0.4032026752250656),
('basemodel__model__layer1', 390),
('basemodel__model__layer2', 371),
('basemodel__model__layer3', 196),
('basemodel__model__learning_rate', 0.00476952337213749),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.10948726819576624),
('include_settings', True),
('scaler', StandardScaler())
```

In [ ]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=200,
                           epochs=35,
                           model__activation1='selu',
                           model__activation2='sigmoid',
                           model__activation3='sigmoid',
                           model__dropout1=0.38531894797761335, 
                           model__dropout2=0.2713124052618954, 
                           model__dropout3=0.4032026752250656, 
                           model__layer1=390, 
                           model__layer2=371, 
                           model__layer3=196, 
                           model__learning_rate=0.00476952337213749,
                           model__optim=Adam,
                           validation_split=0.10948726819576624, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=200, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f9bff274110>, <keras.callbacks.LambdaCallback object at 0x7f9bff278410>], epochs=35, model=<function create_model at 0x7f9bff2b4dd0>, model__activation1='selu', model__activation2='sigmoid', model__activation3='sigmoid', model__dropout1=0.385318947977...ning_rate=0.00476952337213749, model__loss='mse', model__metrics=[<keras.metrics.metrics.RootMeanSquaredError object at 0x7f9b6a07b090>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f9b6a070690>], model__optim=<class 'keras.optimizers.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.10948726819576624, verbose=0),
                    include_settings=True)

In [ ]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 390)               9750      
                                                                 
 dropout_25 (Dropout)        (None, 390)               0         
                                                                 
 dense_41 (Dense)            (None, 371)               145061    
                                                                 
 dropout_26 (Dropout)        (None, 371)               0         
                                                                 
 dense_42 (Dense)            (None, 196)               72912     
                                                                 
 dropout_27 (Dropout)        (None, 196)               0         
                                                                 
 dense_43 (Dense)            (None, 1)               

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.639,RMSE=-32.315
Finished: 2022-10-30 12:30:08.749556


## Non-Linear RUL

Score: 0.7984364934573063    
Test: 0.
```
('basemodel__batch_size', 130),
('basemodel__epochs', 47),
('basemodel__model__activation1', 'relu'),
('basemodel__model__activation2', 'elu'),
('basemodel__model__activation3', 'selu'),
('basemodel__model__dropout1', 0.6591215870708264),
('basemodel__model__dropout2', 0.3075952959408218),
('basemodel__model__dropout3', 0.3971628411684459),
('basemodel__model__layer1', 444),
('basemodel__model__layer2', 512),
('basemodel__model__layer3', 381),
('basemodel__model__learning_rate', 0.0035332717086087725),
('basemodel__model__optim', keras.optimizer_v2.RMSprop.RMSprop),
('basemodel__validation_split', 0.15776882704708337),
('clip_y', 80),
('include_settings', True),
('scaler', StandardScaler())
```

In [ ]:
CLIP=80

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=130,
                           epochs=47,
                           model__activation1='relu',
                           model__activation2='elu',
                           model__activation3='selu',
                           model__dropout1=0.6591215870708264, 
                           model__dropout2=0.3075952959408218, 
                           model__dropout3=0.3971628411684459, 
                           model__layer1=444, 
                           model__layer2=512, 
                           model__layer3=381,  
                           model__learning_rate=0.0035332717086087725,
                           model__optim=RMSprop,
                           validation_split=0.15776882704708337, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=130, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f9bff274110>, <keras.callbacks.LambdaCallback object at 0x7f9bff278410>], epochs=47, model=<function create_model at 0x7f9bff2b4dd0>, model__activation1='relu', model__activation2='elu', model__activation3='selu', model__dropout1=0.6591215870708264, mo...2717086087725, model__loss='mse', model__metrics=[<keras.metrics.metrics.RootMeanSquaredError object at 0x7f9b6e6cfa50>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f9b6e6d6a90>], model__optim=<class 'keras.optimizers.optimizer_v2.rmsprop.RMSprop'>, print_summary=True, validation_split=0.15776882704708337, verbose=0),
                    clip_y=80, include_settings=True)

In [ ]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 444)               11100     
                                                                 
 dropout_31 (Dropout)        (None, 444)               0         
                                                                 
 dense_49 (Dense)            (None, 512)               227840    
                                                                 
 dropout_32 (Dropout)        (None, 512)               0         
                                                                 
 dense_50 (Dense)            (None, 381)               195453    
                                                                 
 dropout_33 (Dropout)        (None, 381)               0         
                                                                 
 dense_51 (Dense)            (None, 1)               

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.841,RMSE=-11.148
Finished: 2022-10-30 12:34:54.867984


## PolyFeatures + Linear RUL

Score: 0.5825762286758006    
Test: 0.
```
('basemodel__batch_size', 341),
('basemodel__epochs', 34),
('basemodel__model__activation1', 'tanh'),
('basemodel__model__activation2', 'tanh'),
('basemodel__model__activation3', 'selu'),
('basemodel__model__dropout1', 0.7525277426098136),
('basemodel__model__dropout2', 0.19508775114046617),
('basemodel__model__dropout3', 0.1),
('basemodel__model__layer1', 409),
('basemodel__model__layer2', 185),
('basemodel__model__layer3', 306),
('basemodel__model__learning_rate', 0.0023786927659858437),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.10918862929162437),
('include_settings', True),
('poly_degree', 2),
('scaler', StandardScaler())
```

In [ ]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=2,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=341,
                           epochs=34,
                           model__activation1='tanh',
                           model__activation2='tanh',
                           model__activation3='selu',
                           model__dropout1=0.7525277426098136, 
                           model__dropout2=0.19508775114046617, 
                           model__dropout3=0.1, 
                           model__layer1=409, 
                           model__layer2=185, 
                           model__layer3=306, 
                           model__learning_rate=0.0023786927659858437,
                           model__optim=Adam,
                           validation_split=0.10918862929162437, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=341, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f9bff274110>, <keras.callbacks.LambdaCallback object at 0x7f9bff278410>], epochs=34, model=<function create_model at 0x7f9bff2b4dd0>, model__activation1='tanh', model__activation2='tanh', model__activation3='selu', model__dropout1=0.7525277426098136, m...786927659858437, model__loss='mse', model__metrics=[<keras.metrics.metrics.RootMeanSquaredError object at 0x7f9b1bb97710>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f9b1bb8da10>], model__optim=<class 'keras.optimizers.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.10918862929162437, verbose=0),
                    include_settings=True, poly_degree=2)

In [ ]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_52 (Dense)            (None, 409)               132925    
                                                                 
 dropout_34 (Dropout)        (None, 409)               0         
                                                                 
 dense_53 (Dense)            (None, 185)               75850     
                                                                 
 dropout_35 (Dropout)        (None, 185)               0         
                                                                 
 dense_54 (Dense)            (None, 306)               56916     
                                                                 
 dropout_36 (Dropout)        (None, 306)               0         
                                                                 
 dense_55 (Dense)            (None, 1)               

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.591,RMSE=-34.394
Finished: 2022-10-30 12:37:15.996738


## PolyFeatures + Non-Linear RUL


Score: 0.791357319322234   
Test: 0.
```
('basemodel__batch_size', 88),
('basemodel__epochs', 6),
('basemodel__model__activation1', 'selu'),
('basemodel__model__activation2', 'tanh'),
('basemodel__model__activation3', 'selu'),
('basemodel__model__dropout1', 0.34493316079151415),
('basemodel__model__dropout2', 0.45457085201703984),
('basemodel__model__dropout3', 0.5997686385621787),
('basemodel__model__layer1', 102),
('basemodel__model__layer2', 155),
('basemodel__model__layer3', 278),
('basemodel__model__learning_rate', 0.008687869782338611),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.4969688376060647),
('clip_y', 84),
('include_settings', True),
('poly_degree', 2),
('scaler', StandardScaler())
```

In [ ]:
CLIP=84

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=2,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=88,
                           epochs=50,
                           model__activation1='selu',
                           model__activation2='tanh',
                           model__activation3='selu',
                           model__dropout1=0.34493316079151415, 
                           model__dropout2=0.45457085201703984, 
                           model__dropout3=0.5997686385621787, 
                           model__layer1=102, 
                           model__layer2=155, 
                           model__layer3=278, 
                           model__learning_rate=0.008687869782338611,
                           model__optim=Adam,
                           validation_split=0.4969688376060647, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=88, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f9bff274110>, <keras.callbacks.LambdaCallback object at 0x7f9bff278410>], epochs=50, model=<function create_model at 0x7f9bff2b4dd0>, model__activation1='selu', model__activation2='tanh', model__activation3='selu', model__dropout1=0.34493316079151415, m...338611, model__loss='mse', model__metrics=[<keras.metrics.metrics.RootMeanSquaredError object at 0x7f9b15e7b7d0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f9b15e7bb50>], model__optim=<class 'keras.optimizers.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.4969688376060647, verbose=0),
                    clip_y=84, include_settings=True, poly_degree=2)

In [ ]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_60 (Dense)            (None, 102)               33150     
                                                                 
 dropout_40 (Dropout)        (None, 102)               0         
                                                                 
 dense_61 (Dense)            (None, 155)               15965     
                                                                 
 dropout_41 (Dropout)        (None, 155)               0         
                                                                 
 dense_62 (Dense)            (None, 278)               43368     
                                                                 
 dropout_42 (Dropout)        (None, 278)               0         
                                                                 
 dense_63 (Dense)            (None, 1)               

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.785,RMSE=-13.710
Finished: 2022-10-30 12:59:16.704738


# MLP-4 

## Linear RUL


Score: 0.5879369617031984    
Test: 0.
```
('basemodel__batch_size', 319),
('basemodel__epochs', 46),
('basemodel__model__activation1', 'relu'),
('basemodel__model__activation2', 'selu'),
('basemodel__model__activation3', 'selu'),
('basemodel__model__activation4', 'selu'),
('basemodel__model__dropout1', 0.683753934762909),
('basemodel__model__dropout2', 0.5369622665770732),
('basemodel__model__dropout3', 0.658577790408341),
('basemodel__model__dropout4', 0.805585273208486),
('basemodel__model__layer1', 148),
('basemodel__model__layer2', 428),
('basemodel__model__layer3', 512),
('basemodel__model__layer4', 510),
('basemodel__model__learning_rate', 0.000866552772585896),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.1501923387205559),
('include_settings', True),
('scaler', StandardScaler())
```


In [ ]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=319,
                           epochs=46,
                           model__activation1='relu',
                           model__activation2='selu',
                           model__activation3='selu',
                           model__activation4='selu',
                           model__dropout1=0.683753934762909, 
                           model__dropout2=0.5369622665770732, 
                           model__dropout3=0.658577790408341, 
                           model__dropout4=0.805585273208486, 
                           model__layer1=148, 
                           model__layer2=428, 
                           model__layer3=512, 
                           model__layer4=510, 
                           model__learning_rate=0.000866552772585896,
                           model__optim=Adam,
                           validation_split=0.1501923387205559, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=319, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f9bff274110>, <keras.callbacks.LambdaCallback object at 0x7f9bff278410>], epochs=46, model=<function create_model at 0x7f9bff2b4dd0>, model__activation1='relu', model__activation2='selu', model__activation3='selu', model__activation4='selu', model__dro...ning_rate=0.000866552772585896, model__loss='mse', model__metrics=[<keras.metrics.metrics.RootMeanSquaredError object at 0x7f9b15c236d0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f9b15c23890>], model__optim=<class 'keras.optimizers.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1501923387205559, verbose=0),
                    include_settings=True)

In [ ]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_64 (Dense)            (None, 148)               3700      
                                                                 
 dropout_43 (Dropout)        (None, 148)               0         
                                                                 
 dense_65 (Dense)            (None, 428)               63772     
                                                                 
 dropout_44 (Dropout)        (None, 428)               0         
                                                                 
 dense_66 (Dense)            (None, 512)               219648    
                                                                 
 dropout_45 (Dropout)        (None, 512)               0         
                                                                 
 dense_67 (Dense)            (None, 510)             

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.640,RMSE=-32.261
Finished: 2022-10-30 13:04:42.458726


## Non-Linear RUL

Score: 0.8031482586558986    
Test: 0.
```
('basemodel__batch_size', 186),
('basemodel__epochs', 48),
('basemodel__model__activation1', 'elu'),
('basemodel__model__activation2', 'elu'),
('basemodel__model__activation3', 'sigmoid'),
('basemodel__model__activation4', 'relu'),
('basemodel__model__dropout1', 0.3192129242485524),
('basemodel__model__dropout2', 0.8716266059527173),
('basemodel__model__dropout3', 0.9),
('basemodel__model__dropout4', 0.1),
('basemodel__model__layer1', 182),
('basemodel__model__layer2', 505),
('basemodel__model__layer3', 478),
('basemodel__model__layer4', 16),
('basemodel__model__learning_rate', 0.002811460975523156),
('basemodel__model__optim', keras.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.1113203480919406),
('clip_y', 80),
('include_settings', True),
('scaler', StandardScaler())
```

In [ ]:
CLIP=80

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=1,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=186,
                           epochs=48,
                           model__activation1='elu',
                           model__activation2='elu',
                           model__activation3='sigmoid',
                           model__activation4='relu',
                           model__dropout1=0.3192129242485524, 
                           model__dropout2=0.8716266059527173, 
                           model__dropout3=0.9, 
                           model__dropout4=0.1, 
                           model__layer1=182, 
                           model__layer2=505, 
                           model__layer3=478, 
                           model__layer4=16, 
                           model__learning_rate=0.002811460975523156,
                           model__optim=Adam,
                           validation_split=0.1113203480919406, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=186, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f9bff274110>, <keras.callbacks.LambdaCallback object at 0x7f9bff278410>], epochs=48, model=<function create_model at 0x7f9bff2b4dd0>, model__activation1='elu', model__activation2='elu', model__activation3='sigmoid', model__activation4='relu', model__dr...0.002811460975523156, model__loss='mse', model__metrics=[<keras.metrics.metrics.RootMeanSquaredError object at 0x7f9b159fc250>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f9b159fc490>], model__optim=<class 'keras.optimizers.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.1113203480919406, verbose=0),
                    clip_y=80, include_settings=True)

In [ ]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_69 (Dense)            (None, 182)               4550      
                                                                 
 dropout_47 (Dropout)        (None, 182)               0         
                                                                 
 dense_70 (Dense)            (None, 505)               92415     
                                                                 
 dropout_48 (Dropout)        (None, 505)               0         
                                                                 
 dense_71 (Dense)            (None, 478)               241868    
                                                                 
 dropout_49 (Dropout)        (None, 478)               0         
                                                                 
 dense_72 (Dense)            (None, 16)              

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.839,RMSE=-11.215
Finished: 2022-10-30 13:07:57.781519


## PolyFeatures + Linear RUL

Score: 0.5429781834407112    
Test: 0.
```
('basemodel__batch_size', 108),
('basemodel__epochs', 40),
('basemodel__model__activation1', 'sigmoid'),
('basemodel__model__activation2', 'relu'),
('basemodel__model__activation3', 'relu'),
('basemodel__model__activation4', 'sigmoid'),
('basemodel__model__dropout1', 0.2621650470812381),
('basemodel__model__dropout2', 0.1),
('basemodel__model__dropout3', 0.5418530339407519),
('basemodel__model__dropout4', 0.17380420072855723),
('basemodel__model__layer1', 490),
('basemodel__model__layer2', 84),
('basemodel__model__layer3', 16),
('basemodel__model__layer4', 109),
('basemodel__model__learning_rate', 0.004902483950446228),
('basemodel__model__optim', keras.optimizer_v2.rmsprop.RMSprop),
('basemodel__validation_split', 0.4284401069475232),
('include_settings', True),
('poly_degree', 2),
('scaler', StandardScaler())
```


In [ ]:
CLIP=-1

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=2,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=108,
                           epochs=40,
                           model__activation1='sigmoid',
                           model__activation2='relu',
                           model__activation3='relu',
                           model__activation4='sigmoid',
                           model__dropout1=0.2621650470812381, 
                           model__dropout2=0.1, 
                           model__dropout3=0.5418530339407519, 
                           model__dropout4=0.17380420072855723, 
                           model__layer1=490, 
                           model__layer2=84, 
                           model__layer3=16, 
                           model__layer4=109, 
                           model__learning_rate=0.004902483950446228,
                           model__optim=RMSprop,
                           validation_split=0.4284401069475232, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=108, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f9bff274110>, <keras.callbacks.LambdaCallback object at 0x7f9bff278410>], epochs=40, model=<function create_model at 0x7f9bff2b4dd0>, model__activation1='sigmoid', model__activation2='relu', model__activation3='relu', model__activation4='sigmoid', mode...3950446228, model__loss='mse', model__metrics=[<keras.metrics.metrics.RootMeanSquaredError object at 0x7f9b0efb3150>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f9b1582dc50>], model__optim=<class 'keras.optimizers.optimizer_v2.rmsprop.RMSprop'>, print_summary=True, validation_split=0.4284401069475232, verbose=0),
                    include_settings=True, poly_degree=2)

In [ ]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_74 (Dense)            (None, 490)               159250    
                                                                 
 dropout_51 (Dropout)        (None, 490)               0         
                                                                 
 dense_75 (Dense)            (None, 84)                41244     
                                                                 
 dropout_52 (Dropout)        (None, 84)                0         
                                                                 
 dense_76 (Dense)            (None, 16)                1360      
                                                                 
 dropout_53 (Dropout)        (None, 16)                0         
                                                                 
 dense_77 (Dense)            (None, 109)             

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.588,RMSE=-34.518
Finished: 2022-10-30 13:11:05.522486


## PolyFeatures + Non-Linear RUL


Score: 0.7790003439779515  
Test: 0.
```
('basemodel__batch_size', 291),
('basemodel__epochs', 15),
('basemodel__model__activation1', 'selu'),
('basemodel__model__activation2', 'relu'),
('basemodel__model__activation3', 'relu'),
('basemodel__model__activation4', 'elu'),
('basemodel__model__dropout1', 0.43138482141909396),
('basemodel__model__dropout2', 0.1),
('basemodel__model__dropout3', 0.15985910954002355),
('basemodel__model__dropout4', 0.6832796453385808),
('basemodel__model__layer1', 466),
('basemodel__model__layer2', 194),
('basemodel__model__layer3', 345),
('basemodel__model__layer4', 98),
('basemodel__model__learning_rate', 0.005555768696453825),
('basemodel__model__optim',
keras.optimizers.optimizer_v2.adam.Adam),
('basemodel__validation_split', 0.22076215606341174),
('clip_y', 93),
('include_settings', True),
('poly_degree', 3),
('scaler', StandardScaler())
```

In [ ]:
CLIP=93

model = MLPWrapperRegressor(
        clip_y=CLIP, scaler=StandardScaler(), poly_degree=3,
        include_settings=True,
        basemodel=
            KerasRegressor(model=create_model,
                           batch_size=291,
                           epochs=50,
                           model__activation1='selu',
                           model__activation2='relu',
                           model__activation3='relu',
                           model__activation4='elu',
                           model__dropout1=0.43138482141909396, 
                           model__dropout2=0.1, 
                           model__dropout3=0.15985910954002355, 
                           model__dropout4=0.6832796453385808, 
                           model__layer1=466, 
                           model__layer2=194, 
                           model__layer3=345, 
                           model__layer4=98, 
                           model__learning_rate=0.005555768696453825,
                           model__optim=Adam,
                           validation_split=0.22076215606341174, 
                           verbose=0, callbacks=[es, printerCallback],
                           model__metrics=[RMSE(), R2()],
                           model__loss='mse',
                           print_summary=True
                           )
    )
model

MLPWrapperRegressor(basemodel=KerasRegressor(batch_size=291, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f9bff274110>, <keras.callbacks.LambdaCallback object at 0x7f9bff278410>], epochs=50, model=<function create_model at 0x7f9bff2b4dd0>, model__activation1='selu', model__activation2='relu', model__activation3='relu', model__activation4='elu', model__drop...53825, model__loss='mse', model__metrics=[<keras.metrics.metrics.RootMeanSquaredError object at 0x7f9b0f18cf10>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f9b0effea50>], model__optim=<class 'keras.optimizers.optimizer_v2.adam.Adam'>, print_summary=True, validation_split=0.22076215606341174, verbose=0),
                    clip_y=93, include_settings=True, poly_degree=3)

In [ ]:
model.fit(X_train_, y_train)
print("Finished:", datetime.datetime.now())

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_89 (Dense)            (None, 466)               1363050   
                                                                 
 dropout_63 (Dropout)        (None, 466)               0         
                                                                 
 dense_90 (Dense)            (None, 194)               90598     
                                                                 
 dropout_64 (Dropout)        (None, 194)               0         
                                                                 
 dense_91 (Dense)            (None, 345)               67275     
                                                                 
 dropout_65 (Dropout)        (None, 345)               0         
                                                                 
 dense_92 (Dense)            (None, 98)              

In [ ]:
# Scaling and formatting test data
test_sc = scale_test(X_test,model)

# Clipping test labels
reclipped_y = y_test.copy()
if(CLIP > 0): reclipped_y = reclipped_y["RUL"].clip(upper=CLIP, inplace=False)

# Evaluation
eval.show_result(reclipped_y, model.basemodel.predict(test_sc))
print("Finished:", datetime.datetime.now())

R2=0.707,RMSE=-17.804
Finished: 2022-10-30 13:21:46.324982
